# Travel Plan Team

## Prepare models

In [1]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_core.models import UserMessage
from dotenv import load_dotenv
import os

load_dotenv() # Load environment variables from .env file

az_model_client = AzureOpenAIChatCompletionClient(
    model = "gpt-4o-2024-11-20",
    azure_endpoint = os.getenv("AZURE_OPENAI_API_URL"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    azure_deployment = "gpt-4o",
    api_version = "2024-12-01-preview"
)

az_o3_model_client = AzureOpenAIChatCompletionClient(
    model = "o3-mini",
    azure_endpoint = os.getenv("AZURE_OPENAI_API_URL"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    azure_deployment = "o3-mini",
    api_version = "2024-12-01-preview"
)

## Prepare Agents

In [2]:
from autogen_agentchat.agents import AssistantAgent
planner_agent = AssistantAgent(
    "planner_agent",
    model_client=az_o3_model_client,
    description="A helpful assistant that can plan trips.",
    system_message="You are a helpful assistant that can suggest a travel plan for a user based on their request.",
    model_client_stream=True,  # Enable streaming tokens.
)

local_agent = AssistantAgent(
    "local_agent",
    model_client=az_model_client,
    description="A local assistant that can suggest local activities or places to visit.",
    system_message="You are a helpful assistant that can suggest authentic and interesting local activities or places to visit for a user and can utilize any context information provided.",
    model_client_stream=True,  # Enable streaming tokens.
)

language_agent = AssistantAgent(
    "language_agent",
    model_client=az_model_client,
    description="A helpful assistant that can provide language tips for a given destination.",
    system_message="You are a helpful assistant that can review travel plans, providing feedback on important/critical tips about how best to address language or communication challenges for the given destination. If the plan already includes language tips, you can mention that the plan is satisfactory, with rationale.",
    model_client_stream=True,  # Enable streaming tokens.
)

travel_summary_agent = AssistantAgent(
    "travel_summary_agent",
    model_client=az_o3_model_client,
    description="A helpful assistant that can summarize the travel plan.",
    system_message="You are a helpful assistant that can take in all of the suggestions and advice from the other agents and provide a detailed final travel plan. You must ensure that the final plan is integrated and complete. YOUR FINAL RESPONSE MUST BE THE COMPLETE PLAN. When the plan is complete and all perspectives are integrated, you can respond with TERMINATE.",
    model_client_stream=True,  # Enable streaming tokens.
)

## Organize team

In [3]:
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console

termination = TextMentionTermination("TERMINATE")
group_chat = RoundRobinGroupChat(
    [planner_agent, local_agent, language_agent, travel_summary_agent], termination_condition=termination
)
await Console(group_chat.run_stream(task="Plan a 3 day trip to Nich, Franch."))

---------- TextMessage (user) ----------
Plan a 3 day trip to Nich, Franch.
---------- ModelClientStreamingChunkEvent (planner_agent) ----------
Below is a sample three‐day itinerary for a trip to Nice (located on the French Riviera) that balances history, culture, stunning views, and day‐trips to nearby gems. Feel free to modify the plan based on your interests, pace, and travel style!

─────────────────────────────  
Day 1: Discovering Nice’s Heart and Seaside Charm

Morning
• Arrive in Nice and check in to your centrally located accommodation. (Some popular choices include Hotel Rossetti in Vieux Nice or the Mercure Nice Centre Grimaldi for convenience.)
• Begin your day with a stroll along the famous Promenade des Anglais. Enjoy a coffee at one of the seaside cafés while taking in views of the sparkling Mediterranean.

Late Morning – Early Afternoon
• Head into Vieux Nice (the Old Town) to wander through its narrow, colorful streets.
• Explore the Cours Saleya Market (especially li

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Plan a 3 day trip to Nich, Franch.', type='TextMessage'), TextMessage(source='planner_agent', models_usage=RequestUsage(prompt_tokens=0, completion_tokens=0), metadata={}, content='Below is a sample three‐day itinerary for a trip to Nice (located on the French Riviera) that balances history, culture, stunning views, and day‐trips to nearby gems. Feel free to modify the plan based on your interests, pace, and travel style!\n\n─────────────────────────────  \nDay 1: Discovering Nice’s Heart and Seaside Charm\n\nMorning\n• Arrive in Nice and check in to your centrally located accommodation. (Some popular choices include Hotel Rossetti in Vieux Nice or the Mercure Nice Centre Grimaldi for convenience.)\n• Begin your day with a stroll along the famous Promenade des Anglais. Enjoy a coffee at one of the seaside cafés while taking in views of the sparkling Mediterranean.\n\nLate Morning – Early Afternoon\